In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
import matplotlib.pyplot as plt

import wrangle as w
import explore as e
import env

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression, LassoLars
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = w.wrangle_zillow()

In [3]:
df.head()

,bedrooms,bathrooms,sq_feet,tax_value,year_built,tax_amount,fips
0,4,3.5,3100,1023282,1998,11013.72,6059
1,2,1.0,1465,464000,1967,5672.48,6111
2,3,2.0,1243,564778,1962,6488.30,6059
3,4,3.0,2376,145143,1970,1777.51,6037
4,4,3.0,2962,773303,1950,9516.26,6037


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50790 entries, 0 to 50789
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bedrooms    50790 non-null  int64  
 1   bathrooms   50790 non-null  float64
 2   sq_feet     50790 non-null  int64  
 3   tax_value   50790 non-null  int64  
 4   year_built  50790 non-null  int64  
 5   tax_amount  50790 non-null  float64
 6   fips        50790 non-null  int64  
dtypes: float64(2), int64(5)
memory usage: 2.7 MB


In [5]:
train, validate, test = w.split_data(df)

In [6]:
train_scaled, validate_scaled, test_scaled = w.scale_data(train, validate, test)

In [8]:
train_scaled.head(3)

,bedrooms,bathrooms,sq_feet,tax_value,year_built,tax_amount,fips
8096,0.500000,0.333333,0.162985,206483,1983,0.036235,6111
45007,0.333333,0.166667,0.152955,110048,1970,0.023914,6037
43774,0.500000,0.333333,0.171701,82423,1956,0.018175,6059


In [10]:
train_scaled.dtypes

bedrooms      float64
bathrooms     float64
sq_feet       float64
tax_value       int64
year_built      int64
tax_amount    float64
fips            int64
dtype: object

In [ ]:
x_cols = train_scaled[['bedroom']]

In [9]:
e.plot_variable_pairs(train_scaled, train.tax_value)

TypeError: Invalid comparison between dtype=int64 and str